# Segmenting and Clustering Neighborhoods in Toronto (week3)
## For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.<br>

## Section 1
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

<img src= "https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1578182400000&hmac=vmAUKv6llcoJI6lJaacXX39qSByKbUIK1A52yw9h_YU" width="500" height="250"/>

### Codes in next cell insprited by [Parsing HTML Tables in Python with pandas](https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/) on scraping the above table, simply using pandas to read the table into a pandas dataframe

In [10]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !sudo pip insatll lxml, html5lib, BeautifulSoup4

# postal_codes_of_Canada:_M table header Postcode,Borough,Neighborhood
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url, header=0)

### To create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [11]:
# df_list is a list. Following code show the types
print("df_list type= {}, df_list[0]= {}, and df_list[0] shape ={}".format(type(df_list), type(df_list[0]),df_list[0].shape))

# Dataframe object is in df_list[0].  Assign df_list[0] to df the list first five rows
df= df_list[0]
df.head()

df_list type= <class 'list'>, df_list[0]= <class 'pandas.core.frame.DataFrame'>, and df_list[0] shape =(287, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
# storage tempoary df shape
df_now=df.shape[0]

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
# drop Neighborhood rows with "Not assigned" and reset_index
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True,inplace=True)
print("df shape after drop cloumn borough with 'Not assigned' rows = {}, number of rows dropped = {}"\
      .format(df.shape, df_now-df.shape[0]))
#df.head()

df shape after drop cloumn borough with 'Not assigned' rows = (210, 3), number of rows dropped = 77


## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [14]:
# use groupby
df_g = df.groupby('Postcode').agg(lambda x:', '.join(set(x))).reset_index()
df_g.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [15]:
df_g.loc[df_g['Neighborhood']=="Not assigned",'Neighborhood']=df_g.loc[df_g['Neighborhood']=="Not assigned",'Borough']

## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [16]:
df_g.shape

(103, 3)

## Section 2
### we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

!sudo pip install geocoder if not installed
!sudo pip insatll pgeocode if not insatlled (package use in this project)

## Since geocoder respone elaspe time take too long, another [python pgeocode](https://pypi.org/project/pgeocode/) is installed to look up geolocation by zip code.  you can follow the page link for the "pgeocode" api

In [17]:
import pgeocode

# parepare a country and postal list
country='ca' # country Canada
postal_code= list(df_g[['Postcode']].values.tolist())#['M1V', 'M5A'...'M9Z']
nomi = pgeocode.Nominatim(country) 
geo=nomi.query_postal_code(postal_code)

In [18]:
geo.columns

Index(['postal_code', 'country code', 'place_name', 'state_name', 'state_code',
       'county_name', 'county_code', 'community_name', 'community_code',
       'latitude', 'longitude', 'accuracy'],
      dtype='object')

In [21]:
geo_ll=geo[['postal_code','latitude', 'longitude']]
geo_ll.rename(columns={'postal_code': 'Postcode', 'latitude':'Latitude', 'longitude':'Longitude'\
                      }, inplace=True)  # rename cloumn 'postal_code'

# merge geo_ll & df_g
merged_inner = pd.merge(left=df_g,right=geo_ll, left_on='Postcode', right_on='Postcode')
merged_inner.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [22]:
# pgeocode has a shortcoming on M7R postal code.  It cannot located the geo data on it.
merged_inner[merged_inner['Latitude'].isnull()==True]

,Postcode,Borough,Neighborhood,Latitude,Longitude
86,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


### Directly import .cvs geolocation file

In [23]:
url='https://cocl.us/Geospatial_data'
geo_list =  pd.read_csv(url)
geo_list.rename(columns={'Postal Code': 'Postcode'}, inplace=True)  # rename cloumn 'postal_code'
geo_list.head()

# merge geo_list & df_g
merged_inner2 = pd.merge(left=df_g,right=geo_list, left_on='Postcode', right_on='Postcode')
merged_inner2.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Section 3

### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

### 1. Obtain venue in Toronto area

Let import necessary libaries

In [24]:
import folium # map rendering library

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Lookup Toronto geolocation

In [25]:
latitude = merged_inner2['Latitude'][37]
longitude = merged_inner2['Longitude'][37]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.67635739999999, -79.2930312.


Create Toronto area postal MAP. Circle spots are the postal code geo-location. This is not the clustering map!

In [26]:
# create map of Manhattan using latitude and longitude values
map_canada_TorontoRegion = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(merged_inner2['Latitude'], merged_inner2['Longitude'], merged_inner2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada_TorontoRegion)  
    
map_canada_TorontoRegion

In [27]:
merged_inner2.loc[37, 'Neighborhood']

'The Beaches'

Obtain Toronto area only data

In [28]:
Toronto_area=merged_inner2[(merged_inner2['Borough']=='West Toronto') | (merged_inner2['Borough']=='East Toronto')].reset_index()
Toronto_area

,index,Postcode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,76,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
5,77,M6J,West Toronto,"Trinity, Little Portugal",43.647927,-79.419750
6,78,M6K,West Toronto,"Parkdale Village, Brockton, Exhibition Place",43.636847,-79.428191
7,82,M6P,West Toronto,"The Junction South, High Park",43.661608,-79.464763
8,83,M6R,West Toronto,"Roncesvalles, Parkdale",43.648960,-79.456325
9,84,M6S,West Toronto,"Swansea, Runnymede",43.651571,-79.484450


In [29]:
neighborhood_latitude = Toronto_area.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_area.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_area.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


## Before obtaining venues search in Foursquare.com, distance between two postal codes is determined by arbitrary picking two landmarks (school & hotel in this case) to caluclate the approxiate disrance.  Distance between [M6J to M6K walking distance](https://www.google.com/maps/dir/70+Fraser+Ave,+Toronto,+ON+M6K+3E1,+Canada/The+Drake+Hotel,+1150+Queen+St+W,+Toronto,+ON+M6J+1J3,+Canada/@43.6400759,-79.4308989,16z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x882b3507423430a3:0xe157ea4537afea89!2m2!1d-79.4243664!2d43.6377887!1m5!1m1!1s0x882b35001adabff1:0x4bd4cc8717937dde!2m2!1d-79.4246377!2d43.6432048) two landmarks is 1.1 Km (0.6835 Mi). 
### The radius parameter will choose to silghly overlapped the area at 600m.

Define Foursquare Credentials and Version

In [30]:
CLIENT_ID = 'GEZxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxYQZ' # your Foursquare ID
CLIENT_SECRET = 'LXMxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxMRHC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GEZxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxYQZ
CLIENT_SECRET:LXMxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxMRHC


#### Now, let's get the top 100 venues that are in Toronto within a radius of 600 meters.

In [31]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 600 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GEZxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxYQZ&client_secret=LXMxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxMRHC&v=20180605&ll=43.67635739999999,-79.2930312&radius=600&limit=100'

In [140]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e135acdb1cac0001cd9208b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.6817574054, 'lng': -79.28557885738863},
   'sw': {'lat': 43.67095739459999, 'lng': -79.30048354261137}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'dis

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [141]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [142]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,Beaches Bake Shop,Bakery,43.680363,-79.289692
3,The Beech Tree,Gastropub,43.680493,-79.288846
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993


In [143]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [144]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [145]:
# type your answer here

Toronto_venues = getNearbyVenues(names=Toronto_area['Neighborhood'],
                                   latitudes=Toronto_area['Latitude'],
                                   longitudes=Toronto_area['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Dovercourt Village, Dufferin
Trinity, Little Portugal
Brockton, Parkdale Village, Exhibition Place
The Junction South, High Park
Parkdale, Roncesvalles
Runnymede, Swansea
Business Reply Mail Processing Centre 969 Eastern


#### Let's check the size of the resulting dataframe

In [146]:
print(Toronto_venues.shape)
Toronto_venues.head()

(300, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighborhood

In [147]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"Dovercourt Village, Dufferin",18,18,18,18,18,18
"Parkdale, Roncesvalles",13,13,13,13,13,13
"Riverdale, The Danforth West",42,42,42,42,42,42
"Runnymede, Swansea",40,40,40,40,40,40
Studio District,42,42,42,42,42,42
The Beaches,6,6,6,6,6,6
"The Beaches West, India Bazaar",19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues

In [148]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 115 uniques categories.


## 3. Analyze Each Neighborhood

In [149]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pub,Record Shop,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [150]:
Toronto_onehot.shape

(300, 115)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [151]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Malay Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pub,Record Shop,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Brockton, Parkdale Village, Exhibition Place",0.045455,0.000000,0.000000,0.000,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.00000,0.000000,0.045455,0.000000,0.136364,0.00,0.00000,0.00000,0.045455,0.00000,0.000000,0.090909,0.00000,0.000000,0.045455,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00000,0.00,0.000,0.000000,0.000000,0.00,0.00000,0.045455,0.000000,0.000000,0.000000,0.00000,0.000000,0.000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.00000,0.000,0.045455,0.045455,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.045455,0.00000,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.00000,0.000000,0.055556,0.055556,0.000000,0.00,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.055556,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.055556,0.055556,0.000000,0.00000,0.00,0.000,0.000000,0.000000,0.00,0.00000,0.000000,0.055556,0.055556,0.000000,0.00000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.00000,0.055556,0.055556,0.000,0.000000,0.055556,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Dovercourt Village, Dufferin",0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.111111,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.00000,0.000000,0.000000,0.000000,0.055556,0.00,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00

Let's check the new size

In [152]:
Toronto_grouped.shape

(11, 115)

Let's print each neighborhood along with the top 3 most common venues

In [153]:
num_top_venues = 3
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2         Pizza Place  0.06


----Dovercourt Village, Dufferin----
         venue  freq
0       Bakery  0.11
1     Pharmacy  0.11
2  Supermarket  0.11


----Parkdale, Roncesvalles----
              venue  freq
0         Gift Shop  0.15
1  Cuban Restaurant  0.08
2               Bar  0.08


----Riverdale, The Danforth West----
              venue  freq
0  Greek Restaurant  0.21
1       Coffee Shop  0.10
2    Ice Cream Shop  0.07


----Runnymede, Swansea----
         venue  freq
0         Café  0.08
1  Coffee Shop  0.08
2  Pizza Place  0.05


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2  American Restaurant  0.05


----The Beaches----

First, let's write a function to sort the venues in descending order.

In [154]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [155]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Gym,Intersection,Italian Restaurant,Convenience Store,Pet Store
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Brewery,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Garden,Pizza Place,Restaurant
2,"Dovercourt Village, Dufferin",Pharmacy,Bakery,Supermarket,Wine Shop,Brewery,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Recording Studio
3,"Parkdale, Roncesvalles",Gift Shop,Bookstore,Cuban Restaurant,Coffee Shop,Movie Theater,Dessert Shop,Restaurant,Dog Run,Eastern European Restaurant,Breakfast Spot
4,"Riverdale, The Danforth West",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Grocery Store,Indian Restaurant,Bubble Tea Shop,Diner


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [156]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 2, 1, 1, 3, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [157]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_area

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Coffee Shop,Trail,Pub,Health Food Store,Other Great Outdoors,Fruit & Vegetable Store,Eastern European Restaurant,Convenience Store,Cosmetics Shop,Garden
1,41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Grocery Store,Indian Restaurant,Bubble Tea Shop,Diner
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Park,Gym,Liquor Store,Pet Store,Pizza Place,Pub,Italian Restaurant,Burrito Place,Burger Joint
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Gastropub,Park,Pizza Place,Ice Cream Shop,Pet Store
4,76,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,4,Pharmacy,Bakery,Supermarket,Wine Shop,Brewery,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Recording Studio


let's visualize the resulting clusters

In [158]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'],\
                                  Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [159]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0,\
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M6R,-79.456325,0,Gift Shop,Bookstore,Cuban Restaurant,Coffee Shop,Movie Theater,Dessert Shop,Restaurant,Dog Run,Eastern European Restaurant,Breakfast Spot


#### Cluster 2

In [160]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1,\
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M4L,-79.315572,1,Sandwich Place,Park,Gym,Liquor Store,Pet Store,Pizza Place,Pub,Italian Restaurant,Burrito Place,Burger Joint
3,M4M,-79.340923,1,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Gastropub,Park,Pizza Place,Ice Cream Shop,Pet Store
5,M6J,-79.419750,1,Bar,Asian Restaurant,Restaurant,Pizza Place,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Coffee Shop,Ice Cream Shop
6,M6K,-79.428191,1,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Gym,Intersection,Italian Restaurant,Convenience Store,Pet Store
7,M6P,-79.464763,1,Mexican Restaurant,Thai Restaurant,Café,Bar,Grocery Store,Furniture / Home Store,Flea Market,Park,Diner,Discount Store
9,M6S,-79.484450,1,Café,Coffee Shop,Sushi Restaurant,Electronics Store,Italian Restaurant,Pizza Place,Restaurant,Burrito Place,Health Food Store,Pub
10,M7Y,-79.321558,1,Light Rail Station,Yoga Studio,Brewery,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Garden,Pizza Place,Restaurant


#### Cluster 3

In [161]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2,\
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Grocery Store,Indian Restaurant,Bubble Tea Shop,Diner


#### Cluster 4

In [162]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3,\
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.293031,3,Coffee Shop,Trail,Pub,Health Food Store,Other Great Outdoors,Fruit & Vegetable Store,Eastern European Restaurant,Convenience Store,Cosmetics Shop,Garden


#### Cluster 5

In [163]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4,\
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M6H,-79.442259,4,Pharmacy,Bakery,Supermarket,Wine Shop,Brewery,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Recording Studio


## End!